<a href="https://colab.research.google.com/github/Pankaj-2003/Assignment/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

In [40]:
# Time series and ML libraries
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
import joblib
import pickle

In [41]:
df = pd.read_csv('/content/cleaned_dataset.csv')

In [42]:
df['weekend_date'] = pd.to_datetime(df['weekend_date'], format='%Y-%m-%d')

In [43]:
serials_with_gaps = []

for sn in df['SerialNum'].unique():
    serial_data = df[df['SerialNum'] == sn].sort_values('weekend_date').copy()
    # Convert 'weekend_date' to datetime objects
    diffs = serial_data['weekend_date'].diff().dropna()

    if not (diffs == pd.Timedelta(days=7)).all():     # checking if any gap ≠ 7 days
        serials_with_gaps.append(sn)
        print(f"SerialNum {sn} has irregular intervals:")
        print(diffs.value_counts())

SerialNum 3 has irregular intervals:
weekend_date
7 days     103
14 days      2
Name: count, dtype: int64
SerialNum 4 has irregular intervals:
weekend_date
7 days    84
0 days    72
Name: count, dtype: int64
SerialNum 5 has irregular intervals:
weekend_date
7 days     107
14 days      1
Name: count, dtype: int64


In [44]:
# removing duplicates per serial_num if any and filling gaps

def fix_gaps_comprehensive(df):

    fixed_data = []

    for sn in df['SerialNum'].unique():
        serial_data = df[df['SerialNum'] == sn].copy()

        serial_data = serial_data.drop_duplicates(subset=['weekend_date'], keep='first') # Remove duplicates (keep first occurrence)

        start_date = serial_data['weekend_date'].min() # Create complete date range
        end_date = serial_data['weekend_date'].max()
        complete_dates = pd.date_range(start=start_date, end=end_date, freq='W-SAT')


        serial_data = serial_data.set_index('weekend_date') # Reindex to complete date range
        serial_data = serial_data.reindex(complete_dates)
        serial_data['SerialNum'] = sn  # Restore SerialNum

        serial_data = serial_data.fillna(method='ffill')  # forward fill

        serial_data = serial_data.reset_index()
        serial_data.rename(columns={'index': 'weekend_date'}, inplace=True)
        fixed_data.append(serial_data)

    return pd.concat(fixed_data, ignore_index=True)

# feature engg.

In [45]:
def fit_random_forest(X_train, y_train):
    """Fit Random Forest model"""
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model

In [46]:
def calculate_monthly_accuracy(actual, predicted, dates):
    """Calculate monthly accuracy using the given formula"""
    df_temp = pd.DataFrame({
        'actual': actual,
        'predicted': predicted,
        'date': dates
    })
    df_temp['month'] = df_temp['date'].dt.to_period('M')

    monthly_accuracy = {}
    for month in df_temp['month'].unique():
        month_data = df_temp[df_temp['month'] == month]
        actual_sum = month_data['actual'].sum()
        if actual_sum > 0:
            mae_sum = np.abs(month_data['predicted'] - month_data['actual']).sum()
            accuracy = 1 - (mae_sum / actual_sum)
            monthly_accuracy[str(month)] = accuracy
        else:
            monthly_accuracy[str(month)] = 0

    return monthly_accuracy


In [47]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# -------------------------------
# STEP 1: Fix Gaps
# -------------------------------
df_fixed = fix_gaps_comprehensive(df)
df_fixed.drop(["quantity"], axis = 1 , inplace = True)
# -------------------------------
# STEP 2: Feature Engineering
# -------------------------------
def create_features(df_series, lag_features=True):
    df_features = df_series.copy()

    # Time-based features
    df_features['year'] = df_features['weekend_date'].dt.year
    df_features['month'] = df_features['weekend_date'].dt.month
    df_features['quarter'] = df_features['weekend_date'].dt.quarter
    df_features['week_of_year'] = df_features['weekend_date'].dt.isocalendar().week
    df_features['day_of_year'] = df_features['weekend_date'].dt.dayofyear

    # # Cyclical features
    # df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
    # df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)
    # df_features['week_sin'] = np.sin(2 * np.pi * df_features['week_of_year'] / 52)
    # df_features['week_cos'] = np.co1s(2 * np.pi * df_features['week_of_year'] / 52)

    # if lag_features:
    #     group = df_features.groupby('SerialNum')
    #     for lag in [1, 2, 3, 4, 8, 12]:
    #         df_features[f'quantity_lag_{lag}'] = group['quantity'].shift(lag)

    #     for window in [4, 8, 12]:
    #         df_features[f'quantity_roll_mean_{window}'] = group['quantity'].transform(
    #             lambda x: x.rolling(window).mean())
    #         df_features[f'quantity_roll_std_{window}'] = group['quantity'].transform(
    #             lambda x: x.rolling(window).std())

    # Add target
    df_features['target'] = df_features['quantity_median_replaced']
    df_features.drop(["quantity_median_replaced"], axis = 1 , inplace = True)
    return df_features

# -------------------------------
# STEP 3: Encode Categorical Features
# -------------------------------
def encode_categorical_features(df):
    """Encode categorical features"""
    df = df.copy()
    categorical_cols = ['channel', 'brand', 'category', 'sub_category']
    df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
    return df_encoded


In [48]:
# -------------------------------
# STEP 4: Apply Features + Encoding
# -------------------------------
df_features = create_features(df_fixed)
df_encoded = encode_categorical_features(df_features)

In [49]:
df_features

,weekend_date,channel,brand,category,sub_category,SerialNum,is_month_start,is_month_end,year,month,quarter,week_of_year,day_of_year,target
0,2022-06-04,Channel1,B1,Cat2,Sub-Cat2,1,0.0,0.0,2022,6,2,22,155,56.0
1,2022-06-11,Channel1,B1,Cat2,Sub-Cat2,1,0.0,0.0,2022,6,2,23,162,122.0
2,2022-06-18,Channel1,B1,Cat2,Sub-Cat2,1,0.0,0.0,2022,6,2,24,169,102.0
3,2022-06-25,Channel1,B1,Cat2,Sub-Cat2,1,0.0,0.0,2022,6,2,25,176,128.0
4,2022-07-02,Channel1,B1,Cat2,Sub-Cat2,1,0.0,0.0,2022,7,3,26,183,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526,2024-06-01,Channel2,B1,Cat1,Sub-Cat1,5,1.0,0.0,2024,6,2,22,153,243.0
527,2024-06-08,Channel2,B1,Cat1,Sub-Cat1,5,0.0,0.0,2024,6,2,23,160,243.0
528,2024-06-15,Channel2,B1,Cat1,Sub-Cat1,5,0.0,0.0,2024,6,2,24,167,243.0
529,2024-06-22,Channel2,B1,Cat1,Sub-Cat1,5,0.0,0.0,2024,6,2,25,174,243.0


In [50]:

# -------------------------------
# STEP 5: Train-Validation Split (Jun–Aug 2024)
# -------------------------------
validation_start = pd.to_datetime('2024-06-01')
validation_end = pd.to_datetime('2024-08-31')

train_data = df_encoded[df_encoded['weekend_date'] < validation_start].copy()
val_data = df_encoded[(df_encoded['weekend_date'] >= validation_start) &
                      (df_encoded['weekend_date'] <= validation_end)].copy()

In [51]:
train_data

,weekend_date,SerialNum,is_month_start,is_month_end,year,month,quarter,week_of_year,day_of_year,target,channel_Channel2,channel_Channel3,brand_B2,category_Cat2,category_Cat3,sub_category_Sub-Cat2,sub_category_Sub-Cat3,sub_category_Sub-Cat4
0,2022-06-04,1,0.0,0.0,2022,6,2,22,155,56.0,False,False,False,True,False,True,False,False
1,2022-06-11,1,0.0,0.0,2022,6,2,23,162,122.0,False,False,False,True,False,True,False,False
2,2022-06-18,1,0.0,0.0,2022,6,2,24,169,102.0,False,False,False,True,False,True,False,False
3,2022-06-25,1,0.0,0.0,2022,6,2,25,176,128.0,False,False,False,True,False,True,False,False
4,2022-07-02,1,0.0,0.0,2022,7,3,26,183,97.0,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,2024-04-27,5,0.0,0.0,2024,4,2,17,118,54.0,True,False,False,False,False,False,False,False
522,2024-05-04,5,0.0,0.0,2024,5,2,18,125,42.0,True,False,False,False,False,False,False,False
523,2024-05-11,5,0.0,0.0,2024,5,2,19,132,243.0,True,False,False,False,False,False,False,False
524,2024-05-18,5,0.0,0.0,2024,5,2,20,139,243.0,True,False,False,False,False,False,False,False


In [52]:
# -------------------------------
# STEP 6: Define Feature Set
# -------------------------------
drop_cols = ['weekend_date', 'SerialNum', 'target']
feature_cols = [col for col in df_encoded.columns if col not in drop_cols]

X_train = train_data[feature_cols]
y_train = train_data['target']

X_val = val_data[feature_cols]
y_val = val_data['target']
val_dates = val_data['weekend_date']

In [53]:
X_train

,is_month_start,is_month_end,year,month,quarter,week_of_year,day_of_year,channel_Channel2,channel_Channel3,brand_B2,category_Cat2,category_Cat3,sub_category_Sub-Cat2,sub_category_Sub-Cat3,sub_category_Sub-Cat4
0,0.0,0.0,2022,6,2,22,155,False,False,False,True,False,True,False,False
1,0.0,0.0,2022,6,2,23,162,False,False,False,True,False,True,False,False
2,0.0,0.0,2022,6,2,24,169,False,False,False,True,False,True,False,False
3,0.0,0.0,2022,6,2,25,176,False,False,False,True,False,True,False,False
4,0.0,0.0,2022,7,3,26,183,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,0.0,0.0,2024,4,2,17,118,True,False,False,False,False,False,False,False
522,0.0,0.0,2024,5,2,18,125,True,False,False,False,False,False,False,False
523,0.0,0.0,2024,5,2,19,132,True,False,False,False,False,False,False,False
524,0.0,0.0,2024,5,2,20,139,True,False,False,False,False,False,False,False


In [54]:
# -------------------------------
# STEP 7: Train Random Forest
# -------------------------------
def fit_random_forest(X_train, y_train):
    model = RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)
    return model

model = fit_random_forest(X_train, y_train)

# -------------------------------
# STEP 8: Predict on Validation
# -------------------------------
val_preds = model.predict(X_val)

In [55]:
# -------------------------------
# STEP 9: Monthly Accuracy
# -------------------------------
def calculate_monthly_accuracy(actual, predicted, dates):
    df_temp = pd.DataFrame({
        'actual': actual,
        'predicted': predicted,
        'date': dates
    })
    df_temp['month'] = df_temp['date'].dt.to_period('M')

    monthly_accuracy = {}
    for month in df_temp['month'].unique():
        month_data = df_temp[df_temp['month'] == month]
        actual_sum = month_data['actual'].sum()
        if actual_sum > 0:
            mae_sum = np.abs(month_data['predicted'] - month_data['actual']).sum()
            accuracy = 1 - (mae_sum / actual_sum)
            monthly_accuracy[str(month)] = accuracy
        else:
            monthly_accuracy[str(month)] = 0

    return monthly_accuracy

val_accuracy = calculate_monthly_accuracy(y_val.values, val_preds, val_dates)

In [56]:

# -------------------------------
# STEP 10: Print Results
# -------------------------------
print("📊 Monthly Validation Accuracy (Jun–Aug 2024):")
for month, acc in val_accuracy.items():
    print(f"{month}: {acc:.4f}")

📊 Monthly Validation Accuracy (Jun–Aug 2024):
2024-06: 0.7548
2024-07: 0.6530
2024-08: 0.5002


# Prediction on sep-nov

In [57]:
def create_future_dataframe(start_date, end_date, serial_nums):
    future_dates = pd.date_range(start=start_date, end=end_date, freq='W-SAT')  # 'W-SAT' for weekend
    future_df = pd.DataFrame([(date, serial) for date in future_dates for serial in serial_nums],
                             columns=['weekend_date', 'SerialNum'])
    return future_df


In [58]:
serial_nums = df_fixed['SerialNum'].unique()
future_df = create_future_dataframe('2024-09-07', '2024-11-30', serial_nums)


In [59]:
latest_info = df_fixed.sort_values('weekend_date').groupby('SerialNum').last().reset_index()
future_full = future_df.merge(latest_info, on='SerialNum', how='left')
future_full = future_full.rename(columns={'weekend_date_x': 'weekend_date'})
future_full = future_full.drop(columns=['weekend_date_y'])  # optional, only if you don't need it


In [22]:
latest_info

,SerialNum,weekend_date,channel,brand,category,sub_category,quantity_median_replaced,is_month_start,is_month_end
0,1,2024-08-31,Channel1,B1,Cat2,Sub-Cat2,78.0,0.0,1.0
1,2,2024-08-31,Channel2,B2,Cat3,Sub-Cat3,180.0,0.0,1.0
2,3,2024-08-31,Channel3,B1,Cat3,Sub-Cat4,261.0,0.0,1.0
3,4,2024-08-31,Channel2,B1,Cat3,Sub-Cat4,1189.0,0.0,1.0
4,5,2024-06-29,Channel2,B1,Cat1,Sub-Cat1,243.0,0.0,0.0


In [23]:
future_full

,weekend_date,SerialNum,channel,brand,category,sub_category,quantity_median_replaced,is_month_start,is_month_end
0,2024-09-07,1,Channel1,B1,Cat2,Sub-Cat2,78.0,0.0,1.0
1,2024-09-07,2,Channel2,B2,Cat3,Sub-Cat3,180.0,0.0,1.0
2,2024-09-07,3,Channel3,B1,Cat3,Sub-Cat4,261.0,0.0,1.0
3,2024-09-07,4,Channel2,B1,Cat3,Sub-Cat4,1189.0,0.0,1.0
4,2024-09-07,5,Channel2,B1,Cat1,Sub-Cat1,243.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
60,2024-11-30,1,Channel1,B1,Cat2,Sub-Cat2,78.0,0.0,1.0
61,2024-11-30,2,Channel2,B2,Cat3,Sub-Cat3,180.0,0.0,1.0
62,2024-11-30,3,Channel3,B1,Cat3,Sub-Cat4,261.0,0.0,1.0
63,2024-11-30,4,Channel2,B1,Cat3,Sub-Cat4,1189.0,0.0,1.0


In [60]:
# Generate features
future_features = create_features(future_full, lag_features=False)  # no lags, future doesn't have them

# Encode categorical variables
future_encoded = encode_categorical_features(future_features)



In [61]:
X_train

,is_month_start,is_month_end,year,month,quarter,week_of_year,day_of_year,channel_Channel2,channel_Channel3,brand_B2,category_Cat2,category_Cat3,sub_category_Sub-Cat2,sub_category_Sub-Cat3,sub_category_Sub-Cat4
0,0.0,0.0,2022,6,2,22,155,False,False,False,True,False,True,False,False
1,0.0,0.0,2022,6,2,23,162,False,False,False,True,False,True,False,False
2,0.0,0.0,2022,6,2,24,169,False,False,False,True,False,True,False,False
3,0.0,0.0,2022,6,2,25,176,False,False,False,True,False,True,False,False
4,0.0,0.0,2022,7,3,26,183,False,False,False,True,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,0.0,0.0,2024,4,2,17,118,True,False,False,False,False,False,False,False
522,0.0,0.0,2024,5,2,18,125,True,False,False,False,False,False,False,False
523,0.0,0.0,2024,5,2,19,132,True,False,False,False,False,False,False,False
524,0.0,0.0,2024,5,2,20,139,True,False,False,False,False,False,False,False


In [62]:
future_encoded = future_encoded[X_train.columns]

In [63]:
future_encoded

,is_month_start,is_month_end,year,month,quarter,week_of_year,day_of_year,channel_Channel2,channel_Channel3,brand_B2,category_Cat2,category_Cat3,sub_category_Sub-Cat2,sub_category_Sub-Cat3,sub_category_Sub-Cat4
0,0.0,1.0,2024,9,3,36,251,False,False,False,True,False,True,False,False
1,0.0,1.0,2024,9,3,36,251,True,False,True,False,True,False,True,False
2,0.0,1.0,2024,9,3,36,251,False,True,False,False,True,False,False,True
3,0.0,1.0,2024,9,3,36,251,True,False,False,False,True,False,False,True
4,0.0,0.0,2024,9,3,36,251,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,0.0,1.0,2024,11,4,48,335,False,False,False,True,False,True,False,False
61,0.0,1.0,2024,11,4,48,335,True,False,True,False,True,False,True,False
62,0.0,1.0,2024,11,4,48,335,False,True,False,False,True,False,False,True
63,0.0,1.0,2024,11,4,48,335,True,False,False,False,True,False,False,True


In [64]:
future_preds = model.predict(future_encoded)
future_results = future_df.copy()
future_results['predicted_quantity'] = future_preds


In [65]:
future_preds = model.predict(future_encoded)
future_results = future_df.copy()
future_results['predicted_quantity'] = future_preds


In [66]:
# Final forecast table
final_forecast = future_results[['weekend_date', 'SerialNum', 'predicted_quantity']]


In [67]:
final_forecast

,weekend_date,SerialNum,predicted_quantity
0,2024-09-07,1,16.920000
1,2024-09-07,2,158.730526
2,2024-09-07,3,491.426620
3,2024-09-07,4,887.244032
4,2024-09-07,5,201.783281
...,...,...,...
60,2024-11-30,1,17.550000
61,2024-11-30,2,153.989861
62,2024-11-30,3,510.754462
63,2024-11-30,4,1061.027952


In [68]:
import joblib
joblib.dump(model, 'final_random_forest_model.pkl')


['final_random_forest_model.pkl']

In [69]:
final_forecast.to_csv('forecast_sept_nov.csv', index=False)
